## 1. Loading & Cleaning

#### Read in the raw Garmin data. Clean and format. Export to a new cleaned .csv.

In [1]:
# imports
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import PercentFormatter
%matplotlib inline

# display settings
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', 500)

---

### loading

In [2]:
# read the data
# index_col -> set index to 'Date'
# thousands -> recognize '1,000' as an int64 instead of a string
# na_values -> replace any '--' values with NA
# usecols -> only use relevant columns

dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')

df = pd.read_csv('data/data.csv', parse_dates=['Date'], infer_datetime_format=True,
                index_col='Date', thousands=',', na_values='--',
                usecols=['Date', 'Activity Type', 'Distance', 'Calories', 'Time',
                            'Avg HR', 'Max HR', 'Aerobic TE', 'Avg Run Cadence', 'Max Run Cadence',
                            'Avg Pace', 'Best Pace', 'Total Ascent', 'Total Descent',
                            'Avg Stride Length', 'Min Elevation', 'Max Elevation'])

# hide HH:MM:SS
df.index = df.index.normalize()

only loading in certain columns, omitting the following:

* Favorite
* Title
* Avg Vertical Ratio
* Avg Vertical Oscillation
* Avg Ground Contact Time
* Avg Run Cadence.1
* Max Run Cadence.1
* Training Stress Score®
* Avg Power
* Max Power
* Grit
* Flow
* Avg. Swolf
* Avg Stroke Rate
* Total Reps
* Dive Time
* Min Temp
* Surface Interval
* Decompression
* Max Temp
* Avg Pace t
* Best Lap Time
* Number of Laps
* Moving Time
* Elapsed Time


---

### exploring

In [3]:
# take a peek!
df.head(3)

,Activity Type,Distance,Calories,Time,Avg HR,Max HR,Aerobic TE,Avg Run Cadence,Max Run Cadence,Avg Pace,Best Pace,Total Ascent,Total Descent,Avg Stride Length,Min Elevation,Max Elevation
Date,,,,,,,,,,,,,,,,
2022-10-27,Running,6.01,596,00:49:46,164,185,4.2,167,178,8:17,7:34,7.0,10.0,1.17,594.0,604.0
2022-10-26,Running,4.01,362,00:34:26,150,161,3.1,165,175,8:34,7:02,3.0,7.0,1.13,594.0,603.0
2022-10-23,Running,6.38,650,00:54:40,167,176,4.8,165,174,8:34,7:44,7.0,3.0,1.14,594.0,604.0


In [4]:
print(f"Since first getting this watch I completed {df.shape[0]} activities.")
print(f"But my watch allowed for multiple activity types, including {df['Activity Type'].unique()}.")
print(f"{df[(df['Activity Type'] == 'Cycling') | (df['Activity Type'] == 'Other')].shape[0]} of those activities weren't runs.")
print("So let's discard those values...")

Since first getting this watch I completed 789 activities.
But my watch allowed for multiple activity types, including ['Running' 'Other' 'Cycling' 'Treadmill Running'].
26 of those activities weren't runs.
So let's discard those values...


In [5]:
# discard activities that weren't running
dropIndex = df[(df['Activity Type'] == 'Cycling') | (df['Activity Type'] == 'Other')].index
df.drop(dropIndex, inplace=True)

i also only want the data leading up to my 100 miler, which occurred february 5th, 2022. my first recorded run with this watch was april 9th, 2018. so for simplicity, i'll select the data from april 2018 to march 2022.

#### _it took almost four years of training for me to go from someone who __doesn't__ run to completing a 100 mile marathon... big things take time._

In [6]:
df.sort_values('Date', inplace=True)
df = df['2018-04-01':'2022-03-01'].copy(deep=True)

---

### nerding

In [7]:
print(f"It took me {(df.index.max() - df.index.min()).days:,} days to go from zero to running a hundred mile marathon.")
print(f"And I ran {df['Distance'].sum().round(2)} miles in training.")
print(f"Along the way burning {df['Calories'].sum():,} calories.")

It took me 1,398 days to go from zero to running a hundred mile marathon.
And I ran 5551.18 miles in training.
Along the way burning 556,701 calories.


currently the timestamp information is in an unusable format. suss the timestamp stuff (Time, Avg Pace, Best Pace) into a more practicle type format

#### convert HH:MM:SS Time into total seconds

In [8]:
# note that now these are just the total number of seconds...
df['Time_s'] = pd.to_timedelta(df['Time']).dt.total_seconds()

In [9]:
print(f"I completed {int(df['Time_s'].sum() % 3600):,} hours of running to finish my hundred miler.")

I completed 3,169 hours of running to finish my hundred miler.


#### convert MM:SS Avg Pace and Best Pace into total seconds

In [10]:
df['Avg Pace_s'] = pd.to_timedelta('00:' + df['Avg Pace']).dt.total_seconds()
df['Best Pace_s'] = pd.to_timedelta('00:' + df['Best Pace']).dt.total_seconds()

as an aside, it's very common for pacing information to get messed up because of a weak gps signal. for example, running in chicago the signal can be redirected by tall buildings resulting in ultra fast (read, _incorrect_) splits.

#### on some days i completed more than one activity. i'd like to have just one row per day.

In [11]:
df.loc['2018-04-14']

,Activity Type,Distance,Calories,Time,Avg HR,Max HR,Aerobic TE,Avg Run Cadence,Max Run Cadence,Avg Pace,Best Pace,Total Ascent,Total Descent,Avg Stride Length,Min Elevation,Max Elevation,Time_s,Avg Pace_s,Best Pace_s
Date,,,,,,,,,,,,,,,,,,,
2018-04-14,Running,4.72,566,00:40:24,174,185,5.0,168,179,8:34,7:39,133.0,103.0,1.12,NaN,79.0,2424.0,514.0,459.0
2018-04-14,Running,3.81,451,00:32:50,176,184,4.8,161,228,8:37,7:41,95.0,99.0,1.16,NaN,79.0,1970.0,517.0,461.0


if i groupby df.index.date and sum(), this can create problems since not all columns are integers. for example, summing the Avg HR from the above data doesn't make sense and could cause issues in later analysis.

the columns which are safe to groupby and sum are ['Distance', 'Calories', 'Total Ascent', 'Total 'Descent', 'Time_s'] so i'll make a new dataframe with just these called numeric_df

In [12]:
# using .transform() will preserve the index
numeric_df = df[['Distance', 'Calories', 'Total Ascent', 'Total Descent', 'Time_s']].groupby(by=df.index.date).transform('sum').copy(deep=True)

In [13]:
numeric_df.head(3)

,Distance,Calories,Total Ascent,Total Descent,Time_s
Date,,,,,
2018-04-09,8.04,1088,203.0,205.0,4067.0
2018-04-11,5.06,640,373.0,381.0,2725.0
2018-04-12,5.98,704,567.0,557.0,2949.0


and i'll create another dataframe with the other values called str_df.

In [14]:
str_df = df.drop(columns=['Distance', 'Calories', 'Total Ascent', 'Total Descent', 'Time', 'Time_s']).copy(deep=True)

In [15]:
str_df.head(3)

,Activity Type,Avg HR,Max HR,Aerobic TE,Avg Run Cadence,Max Run Cadence,Avg Pace,Best Pace,Avg Stride Length,Min Elevation,Max Elevation,Avg Pace_s,Best Pace_s
Date,,,,,,,,,,,,,
2018-04-09,Running,172,187,5.0,161,175,8:26,6:54,1.19,69.0,139.0,506.0,414.0
2018-04-11,Running,170,188,5.0,157,178,8:59,7:54,1.14,328.0,463.0,539.0,474.0
2018-04-12,Running,176,195,5.0,167,213,8:13,5:07,1.17,365.0,458.0,493.0,307.0


#### export both of these as separate .csv files for processing in the next notebook 2-processing.ipynb

In [16]:
numeric_df.to_csv('data/numeric_data.csv')
str_df.to_csv('data/str_data.csv')